History analysis by core

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import random
import glob
import itertools
from itertools import groupby
import seaborn as sns
import matplotlib.ticker as ticker
import sys
import scipy.stats  as stats
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import MinMaxScaler,StandardScaler
scaler = MinMaxScaler()
import statsmodels.stats as sms
from statsmodels.stats import multitest
from sklearn.cluster import AgglomerativeClustering

In [ ]:
tumor_roi_dat = pd.read_csv('new_annotation_Feb2021_KW.txt', delimiter = "\t")
protein_dat=pd.read_csv('ERCC_IgG_norm.csv')
nuc_count = pd.read_csv('nuc_count.csv')

protein_dat=protein_dat.rename({'Unnamed: 0': 'Protein'}, axis=1) 

all_rois=list(protein_dat.columns)[1:]

tumor_core_roi_dict={}
excluded_rois=[]
for loc in range(len(tumor_roi_dat)):
    cur_row = tumor_roi_dat.iloc[loc]
    roi = cur_row['ROI']
    roi_num=roi.split('_')[1]
    if len(roi_num)==1:
        roi = 'ROI_0'+roi_num
    #print(cur_row)
    if roi in all_rois:
        key = (cur_row['Tumor_ID'],cur_row['Core_ID'])
        if key[0]==21 or key[0]==8:
            excluded_rois.append(roi)
        else:
            if key in tumor_core_roi_dict:
                tumor_core_roi_dict[key].append(roi)
            else:
                tumor_core_roi_dict[key] = [roi]            

tumor_ids=list(tumor_roi_dat['Tumor_ID'])
core_ids=list(tumor_roi_dat['Core_ID'])
fish_cores=list(tumor_roi_dat['FISH_core'])

tumor_core_to_shreya_base={(tumor_ids[i],core_ids[i]):fish_cores[i] for i in range(len(tumor_roi_dat))}
tumor_core_to_shreya={}
for k,v in tumor_core_to_shreya_base.items():
    if k in tumor_core_roi_dict.keys():
        tumor_core_to_shreya[k]=v
all_proteins_cores_shreya=list(tumor_core_to_shreya.values())

tumor_roi_dict={}
for k, rois in tumor_core_roi_dict.items():
    tumor=k[0]
    if tumor in tumor_roi_dict.keys():
        tumor_roi_dict[tumor]=tumor_roi_dict[tumor]+rois
    else:
        tumor_roi_dict[tumor]=rois
            
nuc_count=nuc_count.rename({'Unnamed: 0': 'ROI'}, axis=1) 

core_protein_dfs=[]
for core,rois in tumor_core_roi_dict.items():
    cur_df=protein_dat[rois]
    cur_df.index=protein_dat['Protein']
    tot_nuc_count=nuc_count[nuc_count['ROI'].isin(protein_dat[rois])]['nuclei_count'].sum()
    for roi in rois:
        cur_weight=list(nuc_count[nuc_count['ROI']==roi]['nuclei_count'])[0]/tot_nuc_count
        cur_df[roi]=cur_df[roi]*cur_weight 
    cur_df=cur_df.sum(axis=1).to_frame().rename(columns={0:tumor_core_to_shreya[core]})
    core_protein_dfs.append(cur_df)
protein_dat_by_core=pd.concat(core_protein_dfs,1)

protein_dat_by_core=protein_dat_by_core.T

In [ ]:
h_and_e=pd.read_excel('H_and_E_estimates.xlsx',sheet_name='in')
zero_cores=list(h_and_e[(h_and_e['tumor estimate']==0) &(h_and_e['normal brain']==0)&(h_and_e['hemorrhage']==0)&(h_and_e['necrosis']==0)]['core_ID'])
zero_cores=zero_cores+[26]
h_and_e_mask = h_and_e['core_ID'].isin(zero_cores)
h_and_e_new=h_and_e[~h_and_e_mask]
h_and_e_new=h_and_e_new.reset_index()
h_and_e_new=h_and_e_new.drop('index',1)

histo_df=h_and_e_new[h_and_e_new['core_ID'].isin(list(protein_dat_by_core.index))]

In [ ]:
histo_df.index=histo_df['core_ID']

In [ ]:
MVP_df=histo_df[histo_df['other'].isin(['microvascular proliferation','no normal brain, partial core, early MVP','possibly focal MVP','early MVP','Microvascular proliferation'])]

In [ ]:
MVP_cores=list(MVP_df['core_ID'])

In [ ]:
MVP_mask=histo_df['core_ID'].isin(MVP_cores)

In [ ]:
histo_df_by_core=histo_df.copy()
histo_df_by_core_MVP=histo_df_by_core[MVP_mask]
histo_df_by_core_MVP['MVP']=1
histo_df_by_core_not_MVP=histo_df_by_core[~MVP_mask]
histo_df_by_core_not_MVP['pure tumor']=1*(histo_df_by_core_not_MVP['tumor estimate']==1.)
histo_df_by_core_not_MVP['tumor with hemorrhage']=1*((histo_df_by_core_not_MVP['tumor estimate']>0)&(histo_df_by_core_not_MVP['hemorrhage']>0))
histo_df_by_core_not_MVP['tumor with necrosis']=1*((histo_df_by_core_not_MVP['tumor estimate']>0)&(histo_df_by_core_not_MVP['necrosis']>0))


In [ ]:
histo_df_by_core=histo_df.copy()
histo_df_by_core_MVP=histo_df_by_core[MVP_mask]
histo_df_by_core_MVP['MVP']=1
histo_df_by_core_not_MVP=histo_df_by_core[~MVP_mask]
histo_df_by_core_not_MVP['T']=1*(histo_df_by_core_not_MVP['tumor estimate']==1.)
histo_df_by_core_not_MVP['TH']=1*((histo_df_by_core_not_MVP['tumor estimate']>0)&(histo_df_by_core_not_MVP['hemorrhage']>0)&(histo_df_by_core_not_MVP['necrosis']==0))
histo_df_by_core_not_MVP['TN']=1*((histo_df_by_core_not_MVP['tumor estimate']>0)&(histo_df_by_core_not_MVP['necrosis']>0)&(histo_df_by_core_not_MVP['hemorrhage']==0))
histo_df_by_core_not_MVP['TNH']=1*((histo_df_by_core_not_MVP['tumor estimate']>0)&(histo_df_by_core_not_MVP['necrosis']>0)&(histo_df_by_core_not_MVP['hemorrhage']>0))


In [ ]:
new_df=pd.concat([histo_df_by_core_MVP['MVP'].to_frame(),histo_df_by_core_not_MVP['T'].to_frame(),histo_df_by_core_not_MVP['TH'].to_frame(),histo_df_by_core_not_MVP['TN'].to_frame(),histo_df_by_core_not_MVP['TNH'].to_frame()],1)

In [ ]:
new_df=new_df.fillna(0)
other_cores=[10,34,46,80]
keep_cores=set(list(new_df.index)).difference(set(other_cores))
new_df=new_df.loc[keep_cores]

In [ ]:
df_all=pd.concat([new_df,protein_dat_by_core],1).dropna()
df_all['Core']=df_all.index
shreya_to_tumor={v:k[0] for k,v in tumor_core_to_shreya.items()}
df_all['Tumor']=df_all['Core'].map(shreya_to_tumor)

core_to_color={}
label_colors=sns.color_palette("hls")
for core in new_df[new_df['MVP']==1].index:
    core_to_color[core]=label_colors[4]
for core in new_df[new_df['TH']==1].index:
    core_to_color[core]=label_colors[1]
for core in new_df[new_df['TN']==1].index:
    core_to_color[core]=label_colors[2]
for core in new_df[new_df['TNH']==1].index:
    core_to_color[core]=label_colors[3]
for core in new_df[new_df['T']==1].index:
    core_to_color[core]=label_colors[0]
    
color_rows=pd.Series(core_to_color,name='color')

protein_dat_by_core_labeled=pd.concat([protein_dat_by_core,color_rows.to_frame()],1)

protein_dat_by_core_labeled=protein_dat_by_core_labeled.dropna()

protein_dat_by_core_labeled=protein_dat_by_core_labeled.sort_values(by='color')

In [ ]:
plt.rcParams["font.size"] = 8
plt.rcParams["font.family"] = 'Arial'
cur_df0=protein_dat_by_core_labeled
cur_df0=cur_df0.drop(['color'],1)
scaler=MinMaxScaler()
cur_df = pd.DataFrame(scaler.fit_transform(cur_df0), columns=cur_df0.columns)
cur_df.index=cur_df0.index
sns.clustermap(cur_df,method='ward',xticklabels=1,yticklabels=False,cmap="coolwarm",row_colors=[protein_dat_by_core_labeled['color'].dropna()])
#plt.savefig('histology_categories_with_protein_clusters_shown.pdf')
plt.show()

In [ ]:
plt.rcParams["font.size"] = 14
plt.rcParams["font.family"] = 'Arial'
palette = dict(zip(['Pure tumor','Tumor with hemorrhage','Tumor with necrosis','Tumor with hemorrhage and necrosis','Microvascular proliferation'], [label_colors[i] for i in range(5)]))
handles = [mpl.patches.Patch(color=palette[x], label=x) for x in palette.keys()]
plt.legend(handles=handles)
plt.gca().set_axis_off()
plt.tight_layout()
#plt.savefig('histology_clusterplot_legend.pdf')
plt.show()

In [ ]:
clinical=pd.read_excel('GLIOMA_clinical.xlsx',sheet_name='Sheet1')

In [ ]:
cores_with_steroids=[]
tumors_with_steroids=list(clinical[clinical['Steroids ']=='Y']['Patient #'])
for k,v in shreya_to_tumor.items():
    if v in tumors_with_steroids:
        cores_with_steroids.append(k)

In [ ]:
cores_with_steroids

In [ ]:
steroid_dict={}
tumors_with_steroids=list(clinical[clinical['Steroids ']=='Y']['Patient #'])
for k,v in shreya_to_tumor.items():
    if v in tumors_with_steroids:
        steroid_dict[k]='pink'
    else:
        steroid_dict[k]='gray'

In [ ]:
steroids_df_by_core=pd.DataFrame.from_dict(steroid_dict,orient='index').rename(columns={0:'treat_steroids'})


In [ ]:
full_df=pd.concat([protein_dat_by_core_labeled,steroids_df_by_core],1).dropna()

In [ ]:
plt.rcParams["font.size"] = 8
plt.rcParams["font.family"] = 'Arial'
cur_df0=full_df
cur_df0=cur_df0.drop(['color','treat_steroids'],1)
scaler=MinMaxScaler()
cur_df = pd.DataFrame(scaler.fit_transform(cur_df0), columns=cur_df0.columns)
cur_df.index=cur_df0.index
sns.clustermap(cur_df,method='ward',xticklabels=1,yticklabels=False,cmap="coolwarm",row_colors=[full_df['color'],full_df['treat_steroids']])
#plt.savefig('histology_categories_with_protein_clusters_shown_and_steroids.pdf')
plt.show()